In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord

import utils
import model_spectra as NN
import fitting

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [2]:
# read in the standard wavelength grid onto which we interpolate spectra.
wavelength = utils.load_wavelength_array()

# read in all individual neural networks we'll need. 
NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')

# Generate a spectrum from "standard" labels and NN
feh = 0.33
alpha = 0.23
alphafe = alpha - feh
Teff = 5663.6
logg = 4.3
dv = 0.0
real_labels = np.array([alphafe, alpha, alpha, alpha, alpha, alpha, alpha,
                        feh, Teff, logg, dv])
real_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)
data_spec = real_spec + 0.01 * np.random.randn(len(real_spec))
spec_err = 0.01 * np.ones(len(wavelength))

# Apply Mask
kirby_2008_stellar = utils.get_spectral_mask_dict(name='kirby_2008_stellar')
mask = utils.generate_mask_from_dict(**kirby_2008_stellar)
spec_err[mask] = 1e16

# Fit spectrum
popt, pcov, model_spec \
    = fitting.fit_normalized_spectrum_single_star_model(norm_spec = data_spec,
                                                        spec_err = spec_err,
                                                        NN_coeffs = NN_coeffs,
                                                        p0 = None, num_p0 = 1)

def plot(lambda_bounds):
    # zoom in on a small region of the spectrum so we can see what's going on.
    lambda_min = lambda_bounds[0]
    lambda_max = lambda_bounds[1]
    m = (wavelength < lambda_max) & (wavelength > lambda_min)
    
    fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(14, 4))
    
    ax1.plot(wavelength[m], data_spec[m], 'k', lw=0.5, label = r'$\mathrm{Data\ Spectra}$')
    ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.75, label = r'$\mathrm{Best-Fit\ Model}$')
    ax1.set_xlim(lambda_min, lambda_max)
    ax1.set_ylim(0.70,1.10)
    ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
    ax1.legend(loc = 'best', frameon = False, fontsize = 12)
    
    ax2.plot(wavelength[m], data_spec[m]-model_spec[m], 'k', lw=0.5)
    ax2.hlines(0, lambda_min, lambda_max, linestyles='-.')
    ax2.set_ylim(-0.15,0.15)
    ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
    ax2.set_ylabel(r'$\mathrm{Residuals}$')
    
    fig.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    plt.show()
interact(plot, lambda_bounds=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))

print('Best Fit Labels:')
print(popt)

print('True Labels:')
print(real_labels)

interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

Best Fit Labels:
[-0.11 0.21 0.22 0.22 0.19 0.24 0.26 0.35 5672.14 4.30 -0.00]
True Labels:
[-0.10 0.23 0.23 0.23 0.23 0.23 0.23 0.33 5663.60 4.30 0.00]


In [6]:
# Restore Observed spectra
D_PayneDir = utils.D_PayneDir
SpectraDir = D_PayneDir + 'spectra/obs_spectra/'
SpectraFile = 'm15_Horne.npz'
temp = np.load(SpectraDir + SpectraFile)
obj = temp['obj']
spec = temp['spec']
spectral_err = temp['spec_err']
dv = temp['dv']
RA_Dec = SkyCoord(temp['RA_Dec'])
temp.close()


# Standard Labels from APOGEE
matches = [8, 26]
feh = [0.33, -1.26]
alpha = [0.23, -0.36]
Teff = [5663.6, 5650.5]
logg = [4.3, 3.64]
dv = [0.0, 0.0]


i = 0
j = matches[i]


# read in the standard wavelength grid onto which we interpolate spectra.
wavelength = utils.load_wavelength_array()

# read in all individual neural networks we'll need. 
NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')

# Generate a spectrum from "standard" labels and NN
alphafe = alpha[i] - feh[i]
real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                        feh[i], Teff[i], logg[i], dv[i]])
model_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)


# Restore Observed Spectra
norm_spec = spec[j]
spec_err = spectral_err[j]


# Generate Mask
Mask_Cut = 0.04
model_err = norm_spec - model_spec
mask = np.argwhere(np.abs(model_err) > Mask_Cut)
np.save(D_PayneDir + '/other_data/mask.' + obj[j], mask)
spec_err[mask] = 1e16
unmasked_wavelength = wavelength[spec_err < 1e16]
masked_wavelength = wavelength[spec_err == 1e16]

def plot_mask(lambda_bounds):
    # Plot Observed and Model Spectra w/ Mask Overplotted
    lambda_min = lambda_bounds[0]
    lambda_max = lambda_bounds[1]
    m = (wavelength < lambda_max) & (wavelength > lambda_min)
    fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(14, 4))
    ax1.plot(wavelength[m], norm_spec[m], 'k', lw=0.5, label = r'$\mathrm{Observed\ Spectra}$')
    ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.5, label = r'$\mathrm{Model\ Spectra}$')
    ax1.vlines(masked_wavelength, 0.7, 1.1, color='r', alpha=0.5)
    ax1.set_xlim(lambda_min, lambda_max)
    ax1.set_ylim(0.70,1.10)
    ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
    ax1.legend(loc = 'best', frameon = True, fontsize = 12)
    ax2.plot(wavelength[m], norm_spec[m]-model_spec[m], 'k', lw=0.5)
    
    ax2.vlines(masked_wavelength, -0.15, 0.15, color='r', alpha=0.5)
    ax2.hlines(0, lambda_min, lambda_max, linestyles='-')
    ax2.hlines(Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
    ax2.hlines(-Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
    ax2.set_ylim(-0.15,0.15)
    ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
    ax2.set_ylabel(r'$\mathrm{Residuals}$')
    
    plt.suptitle('Obj: %s\n\
                 RA: %.5f Dec: %.5f' % (obj[j], RA_Dec[j].ra.deg, RA_Dec[j].dec.deg))
    fig.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    plt.show()
interact(plot_mask, lambda_bounds=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))


print('Unmasked pixels = %i' % len(unmasked_wavelength))
print('Masked pixels = %i' % len(masked_wavelength))
print('(Includes Telluric mask and masking of blue CCD)')

interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

Unmasked pixels = 5640
Masked pixels = 10610
(Includes Telluric mask and masking of blue CCD)
